Convert the Physics Derivation Graph's v7 JSON format to SQL for variables, units, etc

In [1]:
import json
import pandas
import sqlite3
import time

# data

In [2]:
with open('../../v7_pickle_web_interface/flask/data.json','r') as file_handle:
    data = json.loads(file_handle.read())

In [3]:
data.keys()

dict_keys(['derivations', 'expr local to global', 'expressions', 'inference rules', 'measures', 'operators', 'symbols', 'units'])

The following top-level JSON keys get converted to CSV in this notebook: 
* inference rules
* measures
* operators
* symbols
* units

The top-level JSON keys
* derivations
* expressions
are stored in a property graph

`expr local to global` is made irrelevant by the use of a property graph

# units

In [4]:
data['units'].keys()

dict_keys(['Ampere', 'Farad', 'Kelvin', 'Tesla', 'hand', 'kilogram', 'light-year', 'meter', 'mol', 'parsec', 'second', 'sol'])

In [5]:
list_of_unit_dicts=[]
list_of_unit_references = []
for unit_name, unit_dict in data['units'].items():
    this_unit = {}
    this_unit['unit_name_text'] = unit_name
    #print(unit_dict.keys())
    this_unit["measure"]=unit_dict['measure']
    for name, val in unit_dict['dimensions'].items():
        this_unit[name] = val
    #print(unit_dict['references'])
    for this_ref in eval(str(unit_dict['references'])):
        list_of_unit_references.append({'unit_name_text': unit_name,
                                        'reference_URL': this_ref})
    list_of_unit_dicts.append(this_unit)

In [6]:
df_units = pandas.DataFrame(list_of_unit_dicts)
df_units

,unit_name_text,measure,length,time,mass,temperature,electric charge,amount of substance,luminous intensity
0,Ampere,electric current,0,-1,0,0,1,0,0
1,Farad,capacitance,-2,4,-1,0,2,0,0
2,Kelvin,temperature,0,0,0,1,0,0,0
3,Tesla,magnetic field,0,-1,1,0,-1,0,0
4,hand,length,1,0,0,0,0,0,0
5,kilogram,mass,0,0,1,0,0,0,0
6,light-year,length,1,0,0,0,0,0,0
7,meter,length,1,0,0,0,0,0,0
8,mol,amount of substance,0,0,0,0,0,1,0
9,parsec,length,1,0,0,0,0,0,0


In [7]:
df_unit_references = pandas.DataFrame(list_of_unit_references)
df_unit_references

,unit_name_text,reference_URL
0,Ampere,https://en.wikipedia.org/wiki/Ampere
1,Farad,https://en.wikipedia.org/wiki/Farad
2,Kelvin,https://en.wikipedia.org/wiki/Kelvin
3,Tesla,https://en.wikipedia.org/wiki/Tesla_(unit)
4,hand,https://en.wikipedia.org/wiki/Hand_(unit)
5,kilogram,https://en.wikipedia.org/wiki/Kilogram
6,kilogram,https://www.wikidata.org/wiki/Q11570
7,light-year,https://en.wikipedia.org/wiki/Light-year
8,meter,https://en.wikipedia.org/wiki/Metre
9,mol,https://en.wikipedia.org/wiki/Mole_(unit)


# symbols

In [8]:
list_of_symbol_dicts=[]
list_of_symbol_references = []
list_of_symbol_scope = []
for symbol_id, symbol_dict in data['symbols'].items():
    #print(symbol_id)
    this_symbol={}
    this_symbol['symbol_ID'] = symbol_id
    #print(symbol_dict.keys())
    this_symbol['category'] = symbol_dict['category']
    this_symbol['author'] = symbol_dict['author']
    this_symbol['creation_date'] = symbol_dict['creation date']
    this_symbol['latex'] = symbol_dict['latex']
    if 'references' in this_symbol.keys():
        for this_ref in symbol_dict['references']:
            list_of_symbol_references.append({'symbol_ID': symbol_id, 
                                              'reference_URL': this_ref})

    for name, val in symbol_dict['dimensions'].items():
            this_symbol[name] = val
    if 'name' in this_symbol.keys():
        this_symbol['name'] = symbol_dict['name']
    else:
        this_symbol['name'] = ""
    if isinstance(symbol_dict['scope'], list):
        for this_scope in symbol_dict['scope']:
            list_of_symbol_scope.append({'symbol_ID': symbol_id, 
                                         'scope': this_scope})
    elif isinstance(symbol_dict['scope'], str):
        list_of_symbol_scope.append({'symbol_ID': symbol_id, 
                                     'scope':symbol_dict['scope']})
    else:
        print("ERROR: unrecognized type for ")
        print(symbol_dict['scope'])
    list_of_symbol_dicts.append(this_symbol)

In [9]:
df_symbols = pandas.DataFrame(list_of_symbol_dicts)
df_symbols.head()

,symbol_ID,category,author,creation_date,latex,length,time,mass,temperature,electric charge,amount of substance,luminous intensity,name
0,2321,variable,4a0987466473c589d2c328d12856760e,2020-04-26,\omega,0,-1,0,0,0,0,0,
1,2941,variable,4a0987466473c589d2c328d12856760e,2020-04-26,n_1,0,0,0,0,0,0,0,
2,4929,variable,4a0987466473c589d2c328d12856760e,2020-04-26,KE,2,-2,1,0,0,0,0,
3,4930,variable,4a0987466473c589d2c328d12856760e,2020-04-26,PE,2,-2,1,0,0,0,0,
4,4931,variable,4a0987466473c589d2c328d12856760e,2020-04-26,E,2,-2,1,0,0,0,0,


In [10]:
df_symbol_references = pandas.DataFrame(list_of_symbol_references)
df_symbol_references.empty

True

In [11]:
df_symbol_scope = pandas.DataFrame(list_of_symbol_scope)
df_symbol_scope.head()

,symbol_ID,scope
0,2321,real
1,2941,real
2,4929,real
3,4930,real
4,4931,real


# operators

In [12]:
list_of_operator_dicts=[]
list_of_operator_scope_dicts = []
list_of_operator_macros = []

for operator_name, op_dict in data['operators'].items():
    this_operator_dict = {}
    this_operator_dict['operator_name_text'] = operator_name
    this_operator_dict['integer_argument_count'] = op_dict['argument count']
    this_operator_dict['latex'] = op_dict['latex']

    for this_macro in op_dict['latex macros']:
        list_of_operator_macros.append({'operator_name_text':operator_name, 
                                        'macro': this_macro})
    list_of_operator_dicts.append(this_operator_dict)
    
    for this_scope in op_dict['scope']:
        list_of_operator_scope_dicts.append({'operator_name_text':operator_name, 
                                              'scope':this_scope})

In [13]:
df_operators = pandas.DataFrame(list_of_operator_dicts)
df_operators.shape

(18, 3)

In [14]:
df_operators.head()

,operator_name_text,integer_argument_count,latex
0,addition,2,+
1,cosine,1,\cos
2,cross product,2,\times
3,definite integral,4,\int
4,division,2,/


In [15]:
df_operator_scopes = pandas.DataFrame(list_of_operator_scope_dicts)
df_operator_scopes.shape

(43, 2)

In [16]:
df_operator_scopes.head()

,operator_name_text,scope
0,addition,real
1,addition,vector
2,addition,matrix
3,addition,complex
4,cosine,real


In [17]:
df_operator_macros = pandas.DataFrame(list_of_operator_macros)
df_operator_macros.shape

(19, 2)

In [18]:
df_operator_macros.head(10)

,operator_name_text,macro
0,addition,\addition[2]{ #1 + #2}
1,cosine,\cosine[1]{\cos #1}
2,cross product,\times[2]{ #1 \times #2}
3,definite integral,\integralDefinite[4]{ \int_{#1}^{#2} #3 #4}
4,division,\divisionOneLine[2]{ #1 / #2}
5,division,\divisionFrac[2]{ \frac{#1}{#2}
6,dot product,\dotproduct[2]{ #1 \dot #2}
7,element-wise addition,\elementwiseAddition[2]{ #1 + #2}
8,equals,\equals[2]{ #1 = #2}
9,function,\function[1]{ f #1}


# inference rules

In [19]:
list_of_infrule_dicts = []
for infrule_name, infrule_dict in data['inference rules'].items():
    this_infrule_dict = {}
    this_infrule_dict['inference_rule_name'] = infrule_name
    this_infrule_dict['author'] = infrule_dict['author']
    this_infrule_dict['creation_date YYYY-MM-DD'] = infrule_dict['creation date']
    this_infrule_dict['latex'] = infrule_dict['latex']
    this_infrule_dict['notes_latex'] = infrule_dict['notes']
    this_infrule_dict['integer_number_of_feeds'] = infrule_dict['number of feeds']
    this_infrule_dict['integer_number_of_inputs'] = infrule_dict['number of inputs']
    this_infrule_dict['integer_number_of_outputs'] = infrule_dict['number of outputs']
    this_infrule_dict['assumptions_latex'] = infrule_dict['assumptions']
    list_of_infrule_dicts.append(this_infrule_dict)

In [20]:
df_infrules = pandas.DataFrame(list_of_infrule_dicts)
df_infrules.shape

(94, 9)

In [21]:
df_infrules.head()

,inference_rule_name,author,creation_date YYYY-MM-DD,latex,notes_latex,integer_number_of_feeds,integer_number_of_inputs,integer_number_of_outputs,assumptions_latex
0,apply operator to bra,4a0987466473c589d2c328d12856760e,2020-05-31,Apply operator in Eq.~\ref{eq:#1} to bra; yiel...,,0,1,1,quantum: Dirac notation
1,apply operator to ket,4a0987466473c589d2c328d12856760e,2020-05-31,Apply operator in Eq.~\ref{eq:#1} to ket; yiel...,,0,1,1,quantum: Dirac notation
2,X cross both sides by,4a0987466473c589d2c328d12856760e,2020-04-14,Take cross product of $#1$ and Eq.~\ref{eq:#2}...,,1,1,1,linear algebra
3,X dot both sides,4a0987466473c589d2c328d12856760e,2020-04-14,Take inner product of $#1$ with Eq.~\ref{eq:#2...,,1,1,1,linear algebra
4,add X to both sides,4a0987466473c589d2c328d12856760e,2020-04-14,Add $#1$ to both sides of Eq.~\ref{eq:#2}; yie...,,1,1,1,


# measures

In [22]:
list_of_measure_dicts = []
list_of_measure_refences_dict = []
for measure_name, measure_dict in data['measures'].items():
    this_measure = {}
    this_measure['measure_name'] = measure_name
    if 'references' in measure_dict.keys():
        for this_ref in measure_dict['references']:
            list_of_measure_refences_dict.append({'measure_name': measure_name, 
                                                 'reference_URL': this_ref})

In [23]:
df_measures = pandas.DataFrame(list_of_measure_dicts)
df_measures.empty

True

In [24]:
df_measure_ref = pandas.DataFrame(list_of_measure_refences_dict)
df_measure_ref

,measure_name,reference_URL
0,length,https://en.wikipedia.org/wiki/Unit_of_length


# To SQL

In [25]:
pdg_dfs = {
    "units": df_units,
    "unit_references": df_unit_references,
    "symbols": df_symbols,
    "symbol_references": df_symbol_references,
    "symbol_scope": df_symbol_scope,
    "operators": df_operators,
    "operator_scopes": df_operator_scopes,
    "operator_latex macros": df_operator_macros,
    "inference_rules": df_infrules,
    "measures": df_measures,
    "measure_references": df_measure_ref
}

In [26]:
# from https://stackoverflow.com/questions/68705428/saving-multiple-pandas-dataframes-to-sqlite-in-a-single-transaction

!rm pdg_tables.sql

with sqlite3.connect("pdg_tables.sql") as pdg_db:
    for table_name, df in pdg_dfs.items():
        print("table name:", table_name)
        start_time = time.time()
        # https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html
        df.to_sql(table_name, pdg_db, if_exists="replace")
        print("elapsed",round(time.time()-start_time,2),"seconds")

table name: units


/opt/conda/lib/python3.9/site-packages/pandas/core/generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


elapsed 0.37 seconds
table name: unit_references
elapsed 0.15 seconds
table name: symbols
elapsed 0.18 seconds
table name: symbol_references
elapsed 0.08 seconds
table name: symbol_scope
elapsed 0.11 seconds
table name: operators
elapsed 0.15 seconds
table name: operator_scopes
elapsed 0.17 seconds
table name: operator_latex macros
elapsed 0.11 seconds
table name: inference_rules
elapsed 0.17 seconds
table name: measures
elapsed 0.08 seconds
table name: measure_references
elapsed 0.11 seconds
